# Cognitive Skills & Student Performance Dashboard

This notebook generates a synthetic dataset of students, analyzes correlations between cognitive skills and performance, 
builds a simple ML model, clusters students into personas, and saves outputs for a Next.js dashboard.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os

np.random.seed(42)
os.makedirs('outputs', exist_ok=True)


In [ ]:
N = 300
first = ["Aarav","Arjun","Neha","Priya","Rohan","Siddharth","Kavya","Ananya","Vikram","Isha"]
last = ["Shah","Patel","Reddy","Kumar","Singh","Bose","Mehta","Gupta","Nair","Das"]
names = [f"{np.random.choice(first)} {np.random.choice(last)}" for _ in range(N)]
student_id = [f"S{1000+i}" for i in range(N)]
classes = np.random.choice(["6A","6B","7A","7B","8A"], size=N)
comprehension = np.clip(np.random.normal(70, 12, N), 30, 100)
attention = np.clip(np.random.normal(65, 15, N), 20, 100)
focus = np.clip(np.random.normal(68, 14, N), 25, 100)
retention = np.clip(np.random.normal(66, 13, N), 20, 100)
engagement_time = np.clip(np.random.exponential(scale=45, size=N), 5, 240)
assessment_score = (
    0.28*comprehension + 0.25*attention + 0.22*focus + 0.15*retention + 0.1*(engagement_time/2)
) + np.random.normal(0, 6, N)
assessment_score = np.clip(assessment_score, 0, 100)

df = pd.DataFrame({
    'student_id': student_id,
    'name': names,
    'class': classes,
    'comprehension': np.round(comprehension,1),
    'attention': np.round(attention,1),
    'focus': np.round(focus,1),
    'retention': np.round(retention,1),
    'engagement_time': np.round(engagement_time,1),
    'assessment_score': np.round(assessment_score,1)
})
df.head()


In [ ]:
features = ['comprehension','attention','focus','retention','engagement_time']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(X_scaled)
df.to_csv('outputs/synthetic_students_with_clusters.csv', index=False)
df.head()


In [ ]:
corr = df[['comprehension','attention','focus','retention','engagement_time','assessment_score']].corr()
sns.heatmap(corr, annot=True, fmt='.2f')
plt.title('Correlation matrix')
plt.show()


In [ ]:
plt.scatter(df['attention'], df['assessment_score'], alpha=0.6)
plt.xlabel('Attention')
plt.ylabel('Assessment Score')
plt.title('Attention vs Score')
plt.show()


In [ ]:
X = df[features]
y = df['assessment_score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
rmse, r2


In [ ]:
importances = pd.Series(model.feature_importances_, index=features).sort_values(ascending=False)
importances.plot(kind='bar', title='Feature importances')
plt.show()


In [ ]:
df.groupby('cluster')[features + ['assessment_score']].mean()
